In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("/Users/anzarhussain/Documents/House_Price/notebook/Data/gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [5]:
df=df.drop(labels="id", axis=1)

In [6]:
x=df.drop(labels="price",axis=1)
y=df['price']

In [7]:
x.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [8]:
categorical_cols=x.select_dtypes(include="object").columns
numerical_cols = x.select_dtypes(exclude="object").columns

In [33]:
#categorical_cols = ['cut', 'clarity', 'color']

In [70]:
numerical_cols


Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [35]:
categorical_cols

['cut', 'clarity', 'color']

In [ ]:
cut_map= ["Fair",'Good', 'Very Good', 'Premium', 'Ideal']
cut_clarity = ["I1",'SI2', 'SI1','VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
cut_color = ['D','E','F','G','H','I','J']

In [14]:
import sklearn as sk
from sklearn.impute import SimpleImputer # handling missing value
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [36]:
# Numerical Pipeline
numerical_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("Scaler",StandardScaler())
    ]
)

categorical_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ("encoder", OrdinalEncoder(categories=[cut_map,cut_clarity,cut_color])),
        ('scaler',StandardScaler() )
    ]
)

preprocessor = ColumnTransformer([
    ('numerical_pipeline', numerical_pipeline,numerical_cols),
    ('categorical_pipeline',categorical_pipeline, categorical_cols)
])

In [45]:
# Train test split
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(x,y,test_size=0.20, random_state=42)

In [46]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())

In [47]:
X_test = pd.DataFrame(preprocessor.fit_transform(X_test), columns=preprocessor.get_feature_names_out())

In [48]:
X_train.head()


,numerical_pipeline__carat,numerical_pipeline__depth,numerical_pipeline__table,numerical_pipeline__x,numerical_pipeline__y,numerical_pipeline__z,categorical_pipeline__cut,categorical_pipeline__clarity,categorical_pipeline__color
0,-1.016395,-0.204317,0.402608,-1.202472,-1.187395,-1.194148,-0.132842,-0.648950,-0.936018
1,0.882396,0.720758,-0.118536,0.985177,0.941823,1.036109,-0.132842,0.017052,-0.320002
2,1.529711,0.350728,-1.160823,1.426308,1.394848,1.441611,0.872563,0.017052,1.528047
3,1.896523,0.073206,0.923751,1.741402,1.711965,1.702290,-0.132842,-1.314953,1.528047
4,0.450852,1.738340,1.444895,0.562052,0.525040,0.703019,-2.143651,0.017052,0.912031


In [49]:
X_test.head()

,numerical_pipeline__carat,numerical_pipeline__depth,numerical_pipeline__table,numerical_pipeline__x,numerical_pipeline__y,numerical_pipeline__z,categorical_pipeline__cut,categorical_pipeline__clarity,categorical_pipeline__color
0,-0.629310,0.258720,-0.119119,-0.601318,-0.582021,-0.575751,0.874510,-0.650645,-1.543070
1,2.634357,-2.138639,-0.119119,2.143365,2.216858,1.988163,-1.141067,-1.316036,0.306464
2,-1.129739,-1.216578,0.923216,-1.380337,-1.420773,-1.484109,-0.133279,2.010920,-0.926559
3,-1.020950,-0.571135,0.923216,-1.162937,-1.165501,-1.205742,-0.133279,2.010920,1.539487
4,0.871977,0.627545,-0.640287,0.956720,0.995197,1.021201,0.874510,-0.650645,0.922976


In [50]:
print(X_train.shape)
print(X_test.shape)

(154858, 9)
(38715, 9)


In [51]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [52]:
reg = LinearRegression()
reg.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [53]:
reg.coef_

array([ 6421.41602296,  -129.81979941,   -68.93970393, -1772.23808191,
        -409.88800957,   -81.57634354,    70.26517377,   652.41368541,
        -463.94836356])

In [55]:
 reg.intercept_

np.float64(3972.01423239355)

In [56]:
import numpy as np
def eval_model(true,predict):
    mse = mean_squared_error(true, predict)
    rmse = np.sqrt(mse)
    r2 = r2_score(true, predict)
    mae = mean_absolute_error(true, predict)
    
    return (mse, rmse, r2, mae)

In [64]:
#Train multiple models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "ElasticNet Regression": ElasticNet()
}

In [68]:
model_list = []
r2_list = []
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse, rmse, r2, mae = eval_model(y_test, y_pred)


    model_list.append(list(models.keys())[i])
    print("Models performance: ",model_list[i])
    print("MSE",mse)
    print("rmse", rmse)
    print("r2", r2*100)
    print("mae: ", mae)

    print("="*30)
    print("\n")

Models performance:  Linear Regression
MSE 1013198.1526075372
rmse 1006.577444912977
r2 93.73004483372436
mae:  674.0468954216633


Models performance:  Ridge Regression
MSE 1013204.1820653898
rmse 1006.5804399378073
r2 93.73000752174318
mae:  674.0758284754576


Models performance:  Lasso Regression
MSE 1013632.2294959307
rmse 1006.793042037901
r2 93.72735864383945
mae:  675.4193373917277


Models performance:  ElasticNet Regression
MSE 2333985.2298408616
rmse 1527.7385999708397
r2 85.55664288156255
mae:  1065.5872302794962




/Users/anzarhussain/Documents/House_Price/venv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/anzarhussain/Documents/House_Price/venv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/anzarhussain/Documents/House_Price/venv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/Users/anzarhussain/Documents/House_Price/venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/anzarhussain/Documents/House_Price/venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/anzarhussain/Documents/House_Price/venv/lib/python3.13/si

In [69]:
model_list

['Linear Regression',
 'Ridge Regression',
 'Lasso Regression',
 'ElasticNet Regression']